In [81]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [82]:
ht = pd.read_csv("/content/Heart_Disease_Prediction.csv")

In [83]:
ht

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,Absence
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,Absence
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,Absence
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,Absence


In [84]:
ht = ht.dropna()

In [85]:
# Defining features(X) and target variable (y)
X = ht.drop('Heart Disease', axis=1)
y = ht['Heart Disease']

In [86]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test = train_test_split(X,y)
model = SVC()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

print(accuracy_score(y_test,y_pred))



0.7205882352941176


In [87]:
X=np.array(X)
y=np.array(y)

In [88]:
print("Ant Colony Optimization ")
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
class AntColonyOptimization:
    def __init__(self, n_ants, max_iter, alpha=1.0, beta=1.0, rho=0.1, q=0.1):
        self.n_ants = n_ants
        self.max_iter = max_iter
        self.alpha = alpha  # Pheromone importance
        self.beta = beta    # Heuristic information importance
        self.rho = rho      # Pheromone evaporation rate
        self.q = q          # Pheromone deposit factor

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.pheromone = np.ones(n_features)
        best_features = None
        best_score = -np.inf

        for _ in range(self.max_iter):
            ant_features = []
            for _ in range(self.n_ants):
                features = self.select_features(n_features)
                ant_features.append(features)

            ant_scores = []
            for features in ant_features:
                X_subset = X[:,features]
                X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3)
                clf = SVC()
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                score = accuracy_score(y_test, y_pred)
                ant_scores.append(score)

                if score > best_score:
                    best_score = score
                    best_features = features

            self.update_pheromone(ant_features, ant_scores)

        self.selected_features = best_features
        return self

    def select_features(self, n_features):
        prob = (self.pheromone ** self.alpha) * (1.0 / n_features) ** self.beta
        prob /= prob.sum()
        features = np.random.choice(np.arange(n_features), size=np.random.randint(1, n_features), p=prob)
        return features

    def update_pheromone(self, ant_features, ant_scores):
        delta_pheromone = np.zeros_like(self.pheromone)
        for features, score in zip(ant_features, ant_scores):
            for feature in features:
                delta_pheromone[feature] += score

        self.pheromone = (1 - self.rho) * self.pheromone + self.q * delta_pheromone

        # Ensure pheromone remains positive
        self.pheromone[self.pheromone < 0] = 0

        return self





aco_svm = AntColonyOptimization(n_ants=10, max_iter=20)
aco_svm.fit(X, y)
selected_features = aco_svm.selected_features
print("Selected Features:", selected_features)
X_subset = X[:, selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3)
clf = SVC()
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Selected Features: [11 10  8  1]
Accuracy: 0.7777777777777778


In [89]:
print("Cuckoo Search")

from scipy.special import gamma
class CuckooSearch:
    def __init__(self, n_cuckoos, max_iter, pa=0.25, alpha=1.0):
        self.n_cuckoos = n_cuckoos
        self.max_iter = max_iter
        self.pa = pa    # probability of a cuckoo egg being discovered
        self.alpha = alpha  # step size scaling factor

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.best_nest = None
        self.best_score = -np.inf

        nests = np.random.rand(self.n_cuckoos, n_features)
        for _ in range(self.max_iter):
            # Levy flights
            step_size = self.alpha * self.levy_flight(n_features)
            step = step_size * np.random.randn(self.n_cuckoos, n_features)
            new_nests = nests + step

            # Evaluate new solutions
            for i in range(self.n_cuckoos):
                features = np.clip(new_nests[i], 0, 1)
                X_subset = X[:, features > 0.5]
                X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3)
                clf = SVC()
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                score = accuracy_score(y_test, y_pred)

                if score > self.best_score:
                    self.best_score = score
                    self.best_nest = features

                # Levy flight with probability pa
                if np.random.rand() < self.pa:
                    nests[i] += step[i]

        return self

    def levy_flight(self, n):
        beta = 1.5
        sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / (gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.randn() * sigma
        v = np.random.randn()
        step = u / abs(v) ** (1 / beta)
        return step


cs_svm = CuckooSearch(n_cuckoos=10, max_iter=20)
cs_svm.fit(X, y)

selected_features = np.where(cs_svm.best_nest > 0.5)[0]
print("Selected Features:", selected_features)


X_subset = X[:, selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3)
clf = SVC()
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Selected Features: [ 5  9 11 12]
Accuracy: 0.8148148148148148


In [90]:
print("CrowSearchAlgorithm")
class CrowSearchAlgorithm:
    def __init__(self, n_crows, max_iter, pa=0.25, alpha=1.0):
        self.n_crows = n_crows
        self.max_iter = max_iter
        self.pa = pa    # probability of a crow's flight being influenced by an eagle
        self.alpha = alpha  # step size scaling factor

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.best_nest = None
        self.best_score = -np.inf

        nests = np.random.rand(self.n_crows, n_features)
        for _ in range(self.max_iter):
            # Crow's flight
            step_size = self.alpha / np.sqrt(_ + 1)
            step = step_size * np.random.randn(self.n_crows, n_features)
            new_nests = nests + step

            # Evaluate new solutions
            for i in range(self.n_crows):
                features = np.clip(new_nests[i], 0, 1)
                X_subset = X[:, features > 0.5]
                X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3)
                clf = SVC()
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                score = accuracy_score(y_test, y_pred)

                if score > self.best_score:
                    self.best_score = score
                    self.best_nest = features

                # Eagle's attack with probability pa
                if np.random.rand() < self.pa:
                    nests[i] += step[i]

        return self

csa_svm = CrowSearchAlgorithm(n_crows=10, max_iter=20)
csa_svm.fit(X, y)

selected_features = np.where(csa_svm.best_nest > 0.5)[0]
print("Selected Features:", selected_features)


X_subset = X[:, selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3)
clf = SVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Selected Features: [ 6  8  9 10 11 12]
Accuracy: 0.8765432098765432
